In [2]:
import sympy
sympy.init_printing(use_latex='mathjax', use_unicode=True)
from IPython.display import Math

In [42]:
a, m, t = sympy.symbols('a m t', positive=True, real=True)
theta = sympy.symbols('theta', real=True)
a = sympy.Function('a')(t)
x = sympy.Function('x')(a, theta)
p = sympy.Function('p')(a, theta)
f = sympy.Function('f')(a, x)
j = sympy.Function('j')(x, p, theta, a)
E = sympy.Function('E')(a)
J = sympy.integrate(j, a)
lamda = sympy.Function('lambda')(a)
gamma = sympy.Function('gamma')(a)
gamma

γ(a(t))

In [43]:
pos_eq = p - a**2*sympy.diff(x, t)
mom_eq = f - a * sympy.diff(p, t)
dpos_eq = sympy.diff(pos_eq, theta)
dmom_eq = sympy.diff(mom_eq, theta)
dmom_eq

                    2                                                         
       d           ∂                        ∂                           ∂     
- a(t)⋅──(a(t))⋅────────(p(a(t), θ)) + ───────────(f(a(t), x(a(t), θ)))⋅──(x(a
       dt       ∂a(t) ∂θ               ∂x(a(t), θ)                      ∂θ    

        
        
(t), θ))
        

In [44]:
integrand = j + lamda * pos_eq + gamma * mom_eq
dintegrand = sympy.diff(integrand, theta)

sympy.pprint(sympy.simplify(dintegrand.subs(sympy.diff(a,t), a*E)),use_unicode=True)

  ⎛                  2                                                        
  ⎜         2       ∂                        ∂                           ∂    
- ⎜E(a(t))⋅a (t)⋅────────(p(a(t), θ)) - ───────────(f(a(t), x(a(t), θ)))⋅──(x(
  ⎝              ∂a(t) ∂θ               ∂x(a(t), θ)                      ∂θ   

         ⎞           ⎛                  2                                ⎞    
         ⎟           ⎜         3       ∂                   ∂             ⎟    
a(t), θ))⎟⋅γ(a(t)) - ⎜E(a(t))⋅a (t)⋅────────(x(a(t), θ)) - ──(p(a(t), θ))⎟⋅λ(a
         ⎠           ⎝              ∂a(t) ∂θ               ∂θ            ⎠    

                                                                              
            ∂                                          ∂                     ∂
(t)) + ───────────(j(x(a(t), θ), p(a(t), θ), θ, a(t)))⋅──(p(a(t), θ)) + ──────
       ∂p(a(t), θ)                                     ∂θ               ∂x(a(t

                                                 

In [45]:
dpos_by_parts = (lamda*dpos_eq).expand().subs(-lamda*sympy.diff(x, t, theta), sympy.diff(lamda, t)*sympy.diff(x, theta))
dmom_by_parts = (gamma*dmom_eq).expand().subs(-gamma*sympy.diff(p, t, theta), sympy.diff(gamma, t)*sympy.diff(p, theta))
sympy.pprint(dmom_by_parts)

     d          d            ∂                             ∂                  
a(t)⋅──(a(t))⋅─────(γ(a(t)))⋅──(p(a(t), θ)) + γ(a(t))⋅───────────(f(a(t), x(a(
     dt       da(t)          ∂θ                       ∂x(a(t), θ)             

         ∂             
t), θ)))⋅──(x(a(t), θ))
         ∂θ            


In [52]:
dintegrand_parts = (sympy.diff(j, theta) + dpos_by_parts + dmom_by_parts).expand().subs(sympy.diff(a, t), a*E)
x_bkwd_term = dintegrand_parts.collect(sympy.diff(x, theta)).coeff(sympy.diff(x, theta), 1)
p_bkwd_term = dintegrand_parts.collect(sympy.diff(p, theta)).coeff(sympy.diff(p, theta), 1)
remain_bkwd_term = dintegrand_parts.collect(sympy.diff(x, theta)).coeff(sympy.diff(x, theta), 0).collect(sympy.diff(p, theta)).coeff(sympy.diff(p, theta), 0)
print("Backward equation for lambda")
sympy.pprint(x_bkwd_term)
print("Backward equation for gamma")
sympy.pprint(p_bkwd_term)
print("Remaining terms in integral over t")
remain_bkwd_term += (lamda * sympy.diff(x, theta)).subs(a,0)
remain_bkwd_term += (gamma * sympy.diff(p, theta)).subs(a,0)
sympy.pprint(remain_bkwd_term)

Backward equation for lambda
         3      d                           ∂                                 
E(a(t))⋅a (t)⋅─────(λ(a(t))) + γ(a(t))⋅───────────(f(a(t), x(a(t), θ))) + ────
              da(t)                    ∂x(a(t), θ)                        ∂x(a

 ∂                                         
───────(j(x(a(t), θ), p(a(t), θ), θ, a(t)))
(t), θ)                                    
Backward equation for gamma
         2      d                             ∂                               
E(a(t))⋅a (t)⋅─────(γ(a(t))) + λ(a(t)) + ───────────(j(x(a(t), θ), p(a(t), θ),
              da(t)                      ∂p(a(t), θ)                          

          
 θ, a(t)))
          
Remaining terms in integral over t
     d                  d             ⎛ ∂                                     
γ(0)⋅──(p(0, θ)) + λ(0)⋅──(x(0, θ)) + ⎜───(j(x(a(t), θ), p(a(t), θ), ξ₃, a(t))
     dθ                 dθ            ⎝∂ξ₃                                    

 ⎞│    
)⎟│    
 ⎠│ξ₃=θ


In [81]:
gamma_p = sympy.Function("gamma_0")(a)
gamma_p_expr = (-gamma/(E*a**3)*sympy.diff(f, x) - 1/(E*a**3) * sympy.diff(j, x)).expand()
sympy.pprint(gamma_p)
gamma_of_gp = sympy.solve(gamma_p - gamma_p_expr, gamma)[0]
sympy.pprint(gamma_of_gp)
sympy.pprint(x_bkwd_term.subs(gamma, gamma_of_gp).expand().cancel())
sympy.pprint(p_bkwd_term.subs(gamma, gamma_of_gp).expand().simplify().factor())

γ₀(a(t))
 ⎛         3                    ∂                                         ⎞ 
-⎜E(a(t))⋅a (t)⋅γ₀(a(t)) + ───────────(j(x(a(t), θ), p(a(t), θ), θ, a(t)))⎟ 
 ⎝                         ∂x(a(t), θ)                                    ⎠ 
────────────────────────────────────────────────────────────────────────────
                           ∂                                                
                      ───────────(f(a(t), x(a(t), θ)))                      
                      ∂x(a(t), θ)                                           
           3                        3      d           
- E(a(t))⋅a (t)⋅γ₀(a(t)) + E(a(t))⋅a (t)⋅─────(λ(a(t)))
                                         da(t)         
 ⎛                                2                                           
 ⎜   2        5                  ∂                              ∂             
-⎜- E (a(t))⋅a (t)⋅γ₀(a(t))⋅────────────(f(a(t), x(a(t), θ)))⋅─────(x(a(t), θ)
 ⎜                                     2       